# Digital elevation model

Our source: SRTM

In [ ]:
import rasterio
import rasterio.plot

dem = rasterio.open("../../data/srtm/srtm-utm.geotiff")
rasterio.plot.show(dem)

data = dem.read()


## Slope calculation

http://desktop.arcgis.com/en/arcmap/10.3/tools/spatial-analyst-toolbox/how-slope-works.htm

![](http://desktop.arcgis.com/en/arcmap/10.3/tools/spatial-analyst-toolbox/GUID-64B478F2-045A-4DDD-9E0E-5757B9098B1B-web.gif)

```
slope_degrees = ATAN ( √ ([dz/dx]^2 + [dz/dy]^2) ) * 180/PI
```

In [ ]:
import math
import scipy
import scipy.ndimage
import numpy as np
#slope = math.atan(math.sqrt(dz))


fp = np.array([[1, 1, 1],
               [1, 1, 1],
               [1, 1, 1]], np.uint8)

                     
def get_slope(data):
    """
    +-+-+-+
    |a|b|c|
    +-+-+-+
    |d|e|f|
    +-+-+-+
    |g|h|i|
    +-+-+-+
    
    [dz/dx] = ((c + 2f + i) - (a + 2d + g) / (8 * x_cellsize)
    [dz/dy] = ((g + 2h + i) - (a + 2b + c)) / (8 * y_cellsize)
    """
    m = data.reshape(3, 3)
    dz2dx = ((m[0][2] + 2*m[1][1] + m[2][2]) - (m[0][0] + 2*m[1][0] + m[2][0])) / (8 * dem.res[0])
    dz2dy = ((m[2][0] + 2*m[2][1] + m[2][2]) - (m[0][0] + 2*m[0][1] + m[0][2])) / (8 * dem.res[1])

    slope =  math.sqrt((dz2dx)**2 + (dz2dy)**2)
    if slope > 1.5:
        slope = 0
    slope_deeg = math.atan(slope) * 180/math.pi
    
    return slope_deeg

slope = scipy.ndimage.generic_filter(data[0], get_slope, size=(3,3))

rasterio.plot.show(slope)

## Aspect calculation

http://desktop.arcgis.com/en/arcmap/10.3/tools/spatial-analyst-toolbox/how-aspect-works.htm

![](http://desktop.arcgis.com/en/arcmap/10.3/tools/spatial-analyst-toolbox/GUID-A3590481-6E96-4BD8-BF50-25093FD55F00-web.gif)

In [ ]:
def get_aspect(data):
    """
    +-+-+-+
    |a|b|c|
    +-+-+-+
    |d|e|f|
    +-+-+-+
    |g|h|i|
    +-+-+-+
    
    [dz/dx] = ((c + 2f + i) - (a + 2d + g) / 8
    [dz/dy] = ((g + 2h + i) - (a + 2b + c)) / 8
    """
    m = data.reshape(3, 3)
    dz2dx = ((m[0][2] + 2*m[1][1] + m[2][2]) - (m[0][0] + 2*m[1][0] + m[2][0])) / 8
    dz2dy = ((m[2][0] + 2*m[2][1] + m[2][2]) - (m[0][0] + 2*m[0][1] + m[0][2])) / 8
    
    aspect = 180/math.pi * math.atan2 (dz2dy, -1*dz2dx)
    
    if aspect > 360:
        aspect = aspect%360
        
    if aspect < 0:
        aspect = -1
    return aspect

aspect = scipy.ndimage.generic_filter(data[0], get_aspect, size=(3,3))

rasterio.plot.show(aspect)